In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
from copy import deepcopy
import imageio
import numpy as np
import cv2
import matplotlib.pyplot as plt
import keras
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from tqdm import tqdm
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, InputSpec, DepthwiseConv2D, BatchNormalization, Activation, Conv2D, Add, Conv2DTranspose
from tensorflow.keras import backend as K
from tensorflow.keras.backend import int_shape, permute_dimensions
from collections import OrderedDict
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, SeparableConv2D
from tensorflow.keras.initializers import RandomNormal, RandomUniform

Using TensorFlow backend.
C:\Users\olga\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\olga\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\olga\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\olga\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [3]:
def rename_layer(model_config, src_name, dst_name):
    for layer_item in model_config['input_layers']:
        input_condition = layer_item[0] == src_name
        if input_condition:
            # print('Rename Input:: ', layer_item[0])
            layer_item[0] = dst_name

    for layer_item in model_config['output_layers']:
        output_condition = layer_item[0] == src_name
        if output_condition:
            # print('Rename Output:: ', layer_item[0])
            layer_item[0] = dst_name
    #     print(rename_condition,  layer_list[0])

    for layer_item in model_config['layers']:
        name_condition = layer_item['name'] == src_name
        if name_condition:
            # print('Rename Layers:: name', layer_item['name'])
            layer_item['name'] = dst_name

        name_condition = layer_item['config']['name'] == src_name
        if name_condition:
            # print('Rename Layers Config:: config', layer_item['config']['name'])
            layer_item['config']['name'] = dst_name
            #             print('               Rename Layers Config:: config', layer_item['config']['name'])

        if len(layer_item['inbound_nodes']) > 0:
            for item in layer_item['inbound_nodes'][0]:
                inbound_condition = item[0] == src_name
                if inbound_condition:
                    # print('Rename Layers Inbound::', item[0])
                    item[0] = dst_name
    return model_config

In [4]:
DepthwiseConv2D_config_template = {'name': '_node_name',
 'class_name': 'DepthwiseConv2D',
 'config': {'name': '_node_name',
  'trainable': True,
  'dtype': 'float32',
  'kernel_size': (1, 1),
  'strides': (1, 1),
  'padding': 'same',
  'data_format': 'channels_last',
  'dilation_rate': (1, 1),
  'activation': 'linear',
  'use_bias': False,
  'bias_initializer': {'class_name': 'Zeros', 'config': {'dtype': 'float32'}},
  'bias_regularizer': None,
  'activity_regularizer': None,
  'bias_constraint': None,
  'depth_multiplier': 1,
  'depthwise_initializer': {'class_name': 'GlorotUniform',
   'config': {'seed': None, 'dtype': 'float32'}},
  'depthwise_regularizer': None,
  'depthwise_constraint': None},
 'inbound_nodes': [[['_in_node_name', 0, 0, {}]]]}

In [5]:
placeholder = a = Input((32, 32, 3), name='data')
x = BatchNormalization(name='src', 
                       beta_initializer=RandomNormal(),
                       gamma_initializer=RandomNormal(),
                       moving_mean_initializer=RandomNormal(),
                       moving_variance_initializer=RandomUniform(1,2))(placeholder)
# x = Add()([a, x])
src_model = Model(placeholder, x, name='src_model')

placeholder = Input((32, 32, 3), name='data')
x = DepthwiseConv2D((1, 1), padding='same', name='dst')(placeholder)
dst_model = Model(placeholder, x, name='dst_model')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
gamma, beta, mean, var = src_model.get_layer('src').get_weights()
eps = src_model.get_layer('src').get_config()['epsilon']
a = gamma / np.sqrt(var + eps)
weight = a.reshape((1,1,-1,1))
bias = -a*mean + beta
dst_model.get_layer('dst').set_weights([weight, bias])

In [7]:
x_in = np.random.uniform(size=(1,) + dst_model.input_shape[1:])
print(np.abs(dst_model.predict(x_in) - src_model.predict(x_in)).sum())

4.6063215e-06


In [17]:
def transfer_BatchNormalization_DepthwiseConv2D(src_model, dst_model, transfer_rule):
    gamma, beta, mean, var = src_model.get_layer(transfer_rule['src']).get_weights()
    eps = src_model.get_layer(transfer_rule['src']).get_config()['epsilon']
    a = gamma / np.sqrt(var + eps)
    weight = a.reshape((1,1,-1,1))
    bias = -a*mean + beta
    
    dst_model.get_layer(transfer_rule['dst']).set_weights([weight, bias])
    
def detect_transform_BatchNormalization_DepthwiseConv2D(keras_config):
    index_list = []
    for i, item in enumerate(keras_config['layers']):
        if item['class_name'] == 'BatchNormalization':
            index_list.append(i)
    return index_list

def apply_transform_BatchNormalization_DepthwiseConv2D(keras_config):
    index_list = detect_transform_BatchNormalization_DepthwiseConv2D(keras_config)
    weight_transfer_rule_dict = {}
    while len(index_list) > 0:
        i = index_list[0]
        r_layer_config = keras_config['layers'].pop(i)
        i_layer_config = deepcopy(DepthwiseConv2D_config_template)
        
        i_layer_config['inbound_nodes'] = r_layer_config['inbound_nodes']
        i_layer_config['config']['name'] = i_layer_config['name'] = r_layer_config['name']
        i_layer_config['config']['use_bias'] = True
        i_layer_config['config']['kernel_size'] = (1, 1) 
        keras_config['layers'].insert(i, i_layer_config)
        weight_transfer_rule_dict[i_layer_config['name']] = {'transfer_call': transfer_BatchNormalization_DepthwiseConv2D,
                                                             'src': r_layer_config['name'],
                                                             'dst': i_layer_config['name']}       
        index_list = detect_transform_BatchNormalization_DepthwiseConv2D(keras_config)
    return keras_config, weight_transfer_rule_dict

In [18]:
def transfer_weights(src_model, dst_model, weight_transfer_rule_dict):
    for dst_layer in tqdm(dst_model.layers):
        if dst_layer.name in weight_transfer_rule_dict:
            transfer_rule = weight_transfer_rule_dict[dst_layer.name]
            func = transfer_rule['transfer_call']
            func(src_model, dst_model, transfer_rule)
        else:
            src_model.get_layer(dst_layer.name).set_weights(dst_layer.get_weights())

In [19]:
dst_model_config, weight_transfer_rule_dict = apply_transform_BatchNormalization_DepthwiseConv2D(src_model.get_config())

In [20]:
dst_model = Model.from_config(dst_model_config)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [21]:
transfer_weights(src_model, dst_model, weight_transfer_rule_dict)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 42.52it/s]


In [22]:
x_in = np.random.uniform(size=(1,) + dst_model.input_shape[1:])
print(np.abs(dst_model.predict(x_in) - src_model.predict(x_in)).sum())

4.6752393e-06


In [25]:
0.000820159912 < 1e-4

False

In [24]:
10e-4

0.001